In [10]:
# 12 March 2024
# CSC354 – Assignment1 – ML – Concept Learning
# Usama Tufail
# Fa21-BSE-053
# python program (preferably an ipython notebook) to implement Candidate-Elimination algorithm. Give the following training examples as input to the program.

In [9]:
def is_more_general(hypothesis1, hypothesis2):
    more_general_parts = []
    for x, y in zip(hypothesis1, hypothesis2):
        mg = x == "?" or (x != "0" and (x == y or y == "0"))
        more_general_parts.append(mg)
    return all(more_general_parts)

def fulfills_instance(instance, hypothesis):
    return is_more_general(hypothesis, instance)

def minimal_generalization(hypothesis, instance):
    new_hypothesis = list(hypothesis)
    for i in range(len(hypothesis)):
        if not fulfills_instance(instance[i:i+1], hypothesis[i:i+1]):
            new_hypothesis[i] = '?' if hypothesis[i] != '0' else instance[i]
    return [tuple(new_hypothesis)]

def minimal_specialization(hypothesis, attribute_domains, instance):
    results = []
    for i in range(len(hypothesis)):
        if hypothesis[i] == "?":
            for val in attribute_domains[i]:
                if instance[i] != val:
                    new_hypothesis = hypothesis[:i] + (val,) + hypothesis[i+1:]
                    results.append(new_hypothesis)
        elif hypothesis[i] != "0":
            new_hypothesis = hypothesis[:i] + ('0',) + hypothesis[i+1:]
            results.append(new_hypothesis)
    return results

def generate_null_hypothesis(num_attributes):
    return ("?",) * num_attributes

def generate_specific_hypothesis(num_attributes):
    return ('0',) * num_attributes

def attribute_values(examples):
    attribute_domains = []
    num_attributes = len(examples[0]) - 1
    for i in range(num_attributes):
        values = set()
        for example in examples:
            values.add(example[i])
        attribute_domains.append(values)
    return attribute_domains

def candidate_elimination(examples):
    attribute_domains = attribute_values(examples)

    general_hypotheses = set([generate_null_hypothesis(len(attribute_domains))])
    specific_hypotheses = set([generate_specific_hypothesis(len(attribute_domains))])
    i = 0
    print("\nInitial General Hypotheses (G[{0}]):".format(i), general_hypotheses)
    print("\nInitial Specific Hypotheses (S[{0}]):".format(i), specific_hypotheses)

    for example in examples:
        i += 1
        instance, classification = example[:-1], example[-1]
        if classification == 'Y':
            general_hypotheses = {h for h in general_hypotheses if fulfills_instance(instance, h)}
            specific_hypotheses = generalize_specific_hypotheses(instance, general_hypotheses, specific_hypotheses)
        else:
            specific_hypotheses = {h for h in specific_hypotheses if not fulfills_instance(instance, h)}
            general_hypotheses = specialize_general_hypotheses(instance, attribute_domains, general_hypotheses, specific_hypotheses)
        print("\nUpdated General Hypotheses (G[{0}]):".format(i), general_hypotheses)
        print("\nUpdated Specific Hypotheses (S[{0}]):".format(i), specific_hypotheses)
    return

def generalize_specific_hypotheses(instance, general_hypotheses, specific_hypotheses):
    prev_specific_hypotheses = list(specific_hypotheses)
    for s in prev_specific_hypotheses:
        if s not in specific_hypotheses:
            continue
        if not fulfills_instance(instance, s):
            specific_hypotheses.remove(s)
            new_specific_hypotheses = minimal_generalization(s, instance)
            specific_hypotheses.update([h for h in new_specific_hypotheses if any([is_more_general(g,h) for g in general_hypotheses])])
            specific_hypotheses.difference_update([h for h in specific_hypotheses if any([is_more_general(h, h1) for h1 in specific_hypotheses if h != h1])])
    return specific_hypotheses

def specialize_general_hypotheses(instance, attribute_domains, general_hypotheses, specific_hypotheses):
    prev_general_hypotheses = list(general_hypotheses)
    for g in prev_general_hypotheses:
        if g not in general_hypotheses:
            continue
        if fulfills_instance(instance, g):
            general_hypotheses.remove(g)
            new_general_hypotheses = minimal_specialization(g, attribute_domains, instance)
            general_hypotheses.update([h for h in new_general_hypotheses if any([is_more_general(h, s) for s in specific_hypotheses])])
            general_hypotheses.difference_update([h for h in general_hypotheses if any([is_more_general(g1, h) for g1 in general_hypotheses if h != g1])])
    return general_hypotheses

data = [('big', 'red', 'circle', 'N'),
        ('small', 'red', 'triangle', 'N'),
        ('small', 'red', 'circle', 'Y'),
        ('big', 'blue', 'circle', 'N'),
        ('small', 'blue', 'circle', 'Y')]
candidate_elimination(data)



Initial General Hypotheses (G[0]): {('?', '?', '?')}

Initial Specific Hypotheses (S[0]): {('0', '0', '0')}

Updated General Hypotheses (G[1]): {('small', '?', '?'), ('?', '?', 'triangle'), ('?', 'blue', '?')}

Updated Specific Hypotheses (S[1]): {('0', '0', '0')}

Updated General Hypotheses (G[2]): {('big', '?', 'triangle'), ('?', 'blue', '?'), ('small', '?', 'circle')}

Updated Specific Hypotheses (S[2]): {('0', '0', '0')}

Updated General Hypotheses (G[3]): {('small', '?', 'circle')}

Updated Specific Hypotheses (S[3]): {('small', 'red', 'circle')}

Updated General Hypotheses (G[4]): {('small', '?', 'circle')}

Updated Specific Hypotheses (S[4]): {('small', 'red', 'circle')}

Updated General Hypotheses (G[5]): {('small', '?', 'circle')}

Updated Specific Hypotheses (S[5]): {('small', '?', 'circle')}
